# 🎬 AI Video Clipper Studio - Google Colab Edition (GPU Turbo)

Este notebook permite rodar o **AI Video Clipper** usando as GPUs do Google Colab (Tesla T4) para máxima performance.

### 🚀 Vantagens desta versão:
- **Renderização Rápida**: Usa GPU para transcrever 5x mais rápido.
- **Modelos Melhores**: Usa `whisper-large-v3` automaticamente para transcrição perfeita.
- **Cache Inteligente**: Salva os modelos no seu Drive para não baixar toda vez.

### 📝 Como usar:
1. **Upload**: Faça upload da pasta do projeto para o Google Drive.
2. **Conectar**: Rode a célula abaixo para conectar o Drive.
3. **Start**: Instale e rode.

In [ ]:
# @title 1. Conectar Google Drive e Configurar Cache
from google.colab import drive
import os
import sys

drive.mount('/content/drive')

# @markdown --- 
# @markdown ** 👇 DIGITE O CAMINHO DO PROJETO NO DRIVE:**
PROJECT_PATH = "/content/drive/MyDrive/AI-Video-Clipper-Studio-V3" # @param {type:"string"}

# Cache de modelos para não baixar sempre
MODELS_CACHE_DIR = os.path.join(PROJECT_PATH, "models_cache")
os.makedirs(MODELS_CACHE_DIR, exist_ok=True)

if os.path.exists(PROJECT_PATH):
    os.chdir(PROJECT_PATH)
    print(f"✅ Diretório: {os.getcwd()}")
    print(f"📂 Cache de Modelos: {MODELS_CACHE_DIR}")
else:
    print(f"❌ Erro: O caminho '{PROJECT_PATH}' não existe.")

In [ ]:
# @title 2. Instalação Otimizada (GPU Support)

print("⏳ Instalando FFmpeg e ImageMagick...")
!apt-get update -qq && apt-get install -y ffmpeg imagemagick -qq
!sed -i '/<policy domain="path" rights="none" pattern="@*"\/>/d' /etc/ImageMagick-6/policy.xml

print("⏳ Instalando Bibliotecas Python...")
!pip install -r colab_requirements.txt -q
!pip install pyngrok==7.1.6 -q

import torch
if torch.cuda.is_available():
    print(f"🚀 GPU ACELERADA: {torch.cuda.get_device_name(0)}")
    os.environ["WHISPER_MODEL"] = "large-v3" # Força modelo melhor
    os.environ["VIDEO_QUALITY"] = "ultra"
    os.environ["USE_LOCAL_AI"] = "true"
else:
    print("⚠️ GPU não detectada. Performance será limitada.")
print("🧠 Instalando e Iniciando Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
import time

# Iniciar Ollama em background
subprocess.Popen(["ollama", "serve"])
time.sleep(5) # Esperar servidor subir

print("📥 Baixando modelo Llama3 (pode demorar um pouco)...t")
!ollama pull llama3
print("✅ Ollama Pronto!")


In [ ]:
# @title 3. Iniciar Servidor (Acesse o App)

# @markdown Escolha como acessar o App:
TUNNEL_METHOD = "localtunnel" # @param ["localtunnel", "ngrok"]
NGROK_TOKEN = "" # @param {type:"string"}

import threading
import time
from pyngrok import ngrok

# Limpar processos antigos
!pkill -f flask
!pkill -f cloudflared
!pkill -f lt

def run_flask():
    # Overrides de ambiente para garantir performance máxima na nuvem
    env = os.environ.copy()
    env["FLASK_ENV"] = "production"
    !python app.py

flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

print("⏳ Iniciando servidor...")
time.sleep(8)

if TUNNEL_METHOD == "ngrok":
    if not NGROK_TOKEN:
        print("❌ Token Ngrok necessário!")
    else:
        ngrok.set_auth_token(NGROK_TOKEN)
        public_url = ngrok.connect(5000)
        print(f"\n🎉 SEU APP ESTÁ PRONTO: {public_url}\n")

elif TUNNEL_METHOD == "localtunnel":
    print("🔗 Gerando link público...")
    !npm install -g localtunnel > /dev/null 2>&1
    print("⚠️ PASSWORD DO LOCALTUNNEL:")
    !curl -s ipv4.icanhazip.com
    print("\n")
    !lt --port 5000
